In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns; sns.set()
import scipy.stats, scipy.special

In [2]:
# Дано:
years = 9
k1 = 1.8
k2 = 1.24
k3 = 1.15
k4 = k1

tau_r = years * 3.154*10**7

# Списки вероятностей безотказной работы объединенные по закону распределения
p_exp = np.array([0.981, 0.991, 0.994, 0.976, 0.996])
p_norm = np.array([0.978])
p_l_norm = np.array([0.964])

In [3]:
# Функция поиска параметра lambda эксп. закона распределения:
def find_lamb_exp(p_i):
    return (-1/tau_r)*(np.log(p_i))

# Функция поиска времени безотказной работы эксп. закона распределения:
def norm_exp(x_i, lamb_i):
    return (-1/lamb_i)*np.log(x_i)

# Функция поиска параметра sigma норм. закона распределения:
def find_sig_norm(p_i, k):
    return (tau_r - k*tau_r)/scipy.special.ndtri(1-p_i)

# Функция поиска времени безотказной работы норм. закона распределения:
def norm(x_i, sig_i, k):
    return (scipy.special.ndtri(1-x_i))*sig_i + k*tau_r

# Найдем параметр sigma(zi) в MathCad и вычислим остальные параметры для лн.норм. распределения P4i:
sig_zi = 0.30145307552088571247
m_zi = np.log(k1*tau_r)-(sig_zi ** 2 / 2)
sig_ti = ((np.exp(sig_zi ** 2) - 1) * np.exp(2 * m_zi + sig_zi ** 2)) ** (1 / 2)

# Функция поиска времени безотказной работы лог.норм. закона распределения:
def log_norm(x_i):
    return np.exp(scipy.special.ndtri(1-x_i)*sig_zi + m_zi)

In [4]:
# Проверим данные
lamb_exp = find_lamb_exp(p_exp)
# print(lamb_exp)
print('Выполним проверку найденных данных:', end='\n\n')
print('Эксп. закон: Ti при Xi=Pi (Должно быть равно tau_r)', norm_exp(p_exp, lamb_exp), end='\n\n')

print('Норм. закон: Ti при Xi=Pi=0.978 (Должно быть равно tau_r)', norm(p_norm, find_sig_norm(p_norm, k1), k1), end='\n\n')

print('Лог.норм. закон: Ti при Xi=Pi=0.964 (Должно быть равно tau_r)', log_norm(p_l_norm), end='\n\n')

print('tau_r =', tau_r)

Выполним проверку найденных данных:

Эксп. закон: Ti при Xi=Pi (Должно быть равно tau_r) [2.8386e+08 2.8386e+08 2.8386e+08 2.8386e+08 2.8386e+08]

Норм. закон: Ti при Xi=Pi=0.978 (Должно быть равно tau_r) [2.8386e+08]

Лог.норм. закон: Ti при Xi=Pi=0.964 (Должно быть равно tau_r) [2.8386e+08]

tau_r = 283860000.0


In [5]:
# Инициализация генератора
rg = np.random.default_rng()
# Зададим количество итераций:
n = 100000
# Генерация 22 рядов по n итераций случайных величин равномерно распределенных в промежутке [0,1]
j = rg.random((22, n))
# Создадим списки вероятностей безотк.раб. для элементов подгрупп:
p_1 = np.array([0.978])
p_2 = np.array([0.981, 0.981, 0.981, 0.991, 0.994])
p_3 = np.array([0.976, 0.976, 0.976, 0.976, 0.976, 0.976, 0.976, 0.976, 0.976, 0.976, 0.996, 0.996, 0.996])
p_4 = np.array([0.964, 0.964, 0.964])
J_test = pd.DataFrame(j.T)
# J_test.hist()
# print('Можно заметить, что распределение случайных величин равномерно в промежутке [0:1] \
# для каждого элемента. А значит генератор удовлетворяет требованиям.')

In [6]:
# Определим время безотказной работы элементов 1-й подсистемы для всех итераций.

lamb_1 = find_sig_norm(p_1, k1)
pods_1 = norm(j[0,:], lamb_1, k1)

# Проверка на соответствие заданной вероятности безотказной работы:
print('Элемент #1.1:', (pods_1>tau_r).mean())

Элемент #1.1: 0.97789


In [30]:
# Определим время безотказной работы элементов 2-й подсистемы для всех итераций.

lamb_2 = find_lamb_exp(p_2)
pods_2 = np.empty((lamb_2.shape[0], (j.shape[1])))

for k in range(p_2.shape[0]):
      pods_2[k] = norm_exp(j[1+k,:], lamb_2[k])[np.newaxis, :]

np.place(pods_2, (pods_2>tau_r*k2), tau_r*k2)
# Проверка на соответствие заданной вероятности безотказной работы:
for k in range(pods_2.shape[0]):
    print(f'Элемент #2.{k+1}:', (pods_2[k]>tau_r).mean())

Элемент #2.1: 0.98062
Элемент #2.2: 0.98078
Элемент #2.3: 0.98063
Элемент #2.4: 0.99081
Элемент #2.5: 0.99403


In [8]:
# Определим время безотказной работы элементов 3-й подсистемы для всех итераций.

lamb_3 = find_lamb_exp(p_3[:-1])
pods_3 = np.empty((p_3.shape[0], (j.shape[1])))

for k in range(p_3.shape[0]-1):
      pods_3[k] = norm_exp(j[6+k,:], lamb_3[k])[np.newaxis, :]
pods_3[-1] = j[18]

# Проверка на соответствие заданной вероятности безотказной работы:
for k in range(pods_3.shape[0]):
    print(f'Элемент #3.{k+1}:', (pods_3[k]>tau_r).mean())

Элемент #3.1: 0.97569
Элемент #3.2: 0.97632
Элемент #3.3: 0.97603
Элемент #3.4: 0.97595
Элемент #3.5: 0.97628
Элемент #3.6: 0.97644
Элемент #3.7: 0.97519
Элемент #3.8: 0.9762
Элемент #3.9: 0.97594
Элемент #3.10: 0.97608
Элемент #3.11: 0.99588
Элемент #3.12: 0.99597
Элемент #3.13: 0.0


In [9]:
# Определим время безотказной работы элементов 4-й подсистемы для всех итераций.
pods_4 = log_norm(j[-3:])
# Проверка на соответствие заданной вероятности безотказной работы:
for i in range(pods_4.shape[0]): print(f'Элемент #4.{i+1}:',(pods_4[i]>tau_r).mean())

Элемент #4.1: 0.96438
Элемент #4.2: 0.96501
Элемент #4.3: 0.96383


In [10]:
System = pd.DataFrame(np.concatenate((pods_1[np.newaxis], pods_2, pods_3, pods_4), axis=0))
System.rename(index={0: '1.1', 1: '2.1', 2: '2.2', 3: '2.3', 4: '2.3хр', 5: 'П1',
                     6: '3.1', 7: '3.2', 8: '3.3', 9: '3.4', 10: '3.5', 11: '3.6',
                     12: '3.7', 13: '3.8', 14: '3.9', 15: '3.10', 16: '3.9хр', 17: '3.10хр',
                     18: 'П2', 19: '4.1', 20: '4.2', 21: '4.3'}, inplace=True)

In [11]:
# System

In [31]:
(System.T>tau_r).mean()

1.1       0.97789
2.1       0.98062
2.2       0.98078
2.3       0.98063
2.3хр     0.99081
П1        0.99403
3.1       0.97569
3.2       0.97632
3.3       0.97603
3.4       0.97595
3.5       0.97628
3.6       0.97644
3.7       0.97519
3.8       0.97620
3.9       0.97594
3.10      0.97608
3.9хр     0.99588
3.10хр    0.99597
П2        0.00000
4.1       0.96438
4.2       0.96501
4.3       0.96383
dtype: float64

In [13]:
(System.T['П2']<0.996).mean()

0.9956

Переход к работе с алгоритмами.

По-возможности будут использоваться функции и методы библиотеки NumPy, дабы ускорить работу алгоритма (матричные операции много быстрее, в сравнении с циклами).

Создание таблицы для занесения в неё времени работы элементов определяющих минимальное время безотказной работы подсистем 1-4:

In [14]:
T = pd.DataFrame(columns=['T1_j', 'T2_j', 'T3_j', 'T4_j'])

In [32]:
# Занесение в таблицу времени безотказной работы 1-й подсистемы:
T.T1_j = pods_1

In [33]:
# Исполнение алгоритма по поиску времени безотказной работы 4-й подсистемы:
T.T4_j = np.fmax(np.fmin(pods_4[0], pods_4[1]), pods_4[2])

In [34]:
# Исполнение алгоритма по поиску времени безотказной работы 2-й подсистемы:
# Объявляем список А, который удовлетворяет условию: A = MIN{t2.1_j, t2.2_j}:
A = np.fmin(pods_2[0], pods_2[1])

In [35]:
# Опишем правило выделения элементов из списка А, которые: 1) меньше tau_r; 2) для своей итерации имеют t2.3хр_j,
# который БОЛЬШЕ чем соответствующий элемент из списка А. 
# Их в зависимости от дальнейших условий нужно изменить следуя алгоритму.
cond_1 = (tau_r > A) & (pods_2[3] >= A)

In [36]:
# Введём условия для определения (t2.3_j >= tП1_j), и объединим с предыдущим условием.
A_plus_tP = pods_2[2] >= pods_2[4]
A_plus_t23 = np.logical_not(A_plus_tP)

A_plus_t23 = cond_1 & A_plus_t23
A_plus_tP = cond_1 & A_plus_tP

In [37]:
# Используем принятые правила как маску для манипуляций с элементами вектора А:
np.place(A, A_plus_t23, (A[A_plus_t23]+pods_2[2][A_plus_t23]))
np.place(A, A_plus_tP, (A[A_plus_tP]+pods_2[4][A_plus_tP]))

In [23]:
T.T2_j = A

In [24]:
T

,T1_j,T2_j,T3_j,T4_j
0,5.619705e+08,3.287263e+09,NaN,4.402721e+08
1,6.271053e+08,5.529482e+09,NaN,5.438641e+08
2,5.448530e+08,1.864962e+09,NaN,6.876435e+08
3,6.876436e+08,7.489431e+09,NaN,3.850716e+08
4,4.409454e+08,7.542712e+09,NaN,5.975018e+08
...,...,...,...,...
99995,4.464571e+08,1.492386e+09,NaN,5.414064e+08
99996,6.964936e+08,2.445379e+09,NaN,5.063444e+08
99997,3.711271e+08,8.859424e+09,NaN,4.919872e+08
99998,5.161057e+08,5.134953e+09,NaN,4.881280e+08


In [25]:
A.mean()

7821823367.755274

In [26]:
(A>=tau_r).mean()

0.99939

In [38]:
(A>=tau_r).mean()

0.99939

In [28]:
(pods_1 >= (tau_r*k1)).mean()

0.49873

In [39]:
A.mean()

356838991.14670384